In [ ]:
!pip install web3

In [ ]:
import time
import json
import requests
import pandas as pd
from tqdm import tqdm
from web3 import Web3
from web3.exceptions import ABIFunctionNotFound

In [ ]:
ETH_API_KEY = "2CAK8GFHWXBZATUVPDJ7WTAVR98EP7JQXC"
ETH_URL = "https://api.etherscan.io/api"

RPC_URL = "https://rpc.ankr.com/eth/b1bdd2819d3da7be4760d270af7ef985a74725cbe337c348bca633d096806c5a"
w3 = Web3(Web3.HTTPProvider(RPC_URL))

getAccountBalance_url = "https://rpc.ankr.com/multichain/79258ce7f7ee046decc3b5292a24eb4bf7c910d7e39b691384c7ce0cfb839a01/?ankr_getAccountBalance="

In [ ]:
topics = [
    "0x783cca1c0412dd0d695e784568c96da2e9c22ff989357a2e8b1d9b2b4e6b7118"
]

In [ ]:
factories = {
    "pancakeswap": {
        "address" : ["0x0bfbcf9fa4f9c56b0f40a671ad40e0805a091865"],
        "blockInterval" : 5000000,
        },
}

In [ ]:
# Get the number of most recent block
def get_latest_block():
    params = {
        "module": "proxy",
        "action": "eth_blockNumber",
        "apikey": ETH_API_KEY
    }
    response = requests.post(ETH_URL, params=params)
    data = json.loads(response.text)
    latest_block = int(data['result'], 16)
    return latest_block

In [ ]:
def get_abi(address):
    params = {
        "module": "contract",
        "action": "getabi",
        "address": address,
        "apikey": ETH_API_KEY
    }
    response = requests.post(ETH_URL, params=params)
    time.sleep(0.2)
    data = json.loads(response.text)
    result = data['result']
    return result

In [ ]:
abi_templete = get_abi("0xb078bf211e330b5f95b7114ae845188cc36b795d")

def get_contract(address):
    try:
        abi = get_abi(address)
        checksum_address = Web3.to_checksum_address(address)
        contract = w3.eth.contract(address=checksum_address, abi=abi)
        return contract
    except Exception:
        checksum_address = Web3.to_checksum_address(address)
        contract = w3.eth.contract(address=checksum_address, abi=abi_templete)
        return contract

In [ ]:
def get_fee_from_fee(contract, name):
    deno_dict = {
        "pancakeswap": 10000*100,
        "curve": 10**8,
        "uniswap_v3": 10000*100
    }
    if name in deno_dict:
        fee = float(contract.functions.fee().call())
        fee = fee / deno_dict[name]
        return fee
    elif name == "fraxswap":
        fee = 10000 - float(contract.functions.fee().call())
        fee = fee / 10000
        return fee
    return None


# NomiswapStable
def get_fee_from_swapFee(contract, name):
    try:
        swapFee = float(contract.functions.swapFee().call())
        max_fee = float(contract.functions.MAX_FEE().call())
        fee = swapFee / max_fee
        return fee
    except Exception as e:
        return None


def get_fee_from_getSwapFeePercentage(contract, name):
    if name == "balancerv2":
        swapFeePercentage = float(contract.functions.getSwapFeePercentage().call())
        fee = swapFeePercentage / 10**18
        return fee
    return None


# saddle
def get_fee_from_swapStorage(contract, name):
    if name == "saddle":
        swapStorage = contract.functions.swapStorage().call()
        swapFee = float(swapStorage[4])
        fee = swapFee / 10**8
        return fee
    return None


def get_fee_from_tradingFeePercent(contract, name):
    if name == "linkswap":
        tradingFeePercent = float(contract.functions.tradingFeePercent().call())
        fee = tradingFeePercent / 10**6
        return fee
    return None


def get_fee_from_factory(contract, name):
    if name == "crodefiswap":
        factory_address = contract.functions.factory().call()
        factory_source = get_source_code(factory_address)
        factory_contract = get_contract(factory_address, factory_source)
        totalFeeBasisPoint = float(factory_contract.functions.totalFeeBasisPoint().call())
        fee = totalFeeBasisPoint / 10000
        return fee
    return None


def get_fee(contract, name):
    fix_fee_dict = {
        "uniswapv2": 3/1000,
        "sushiswap": 3/1000,
        "verseexchange": 3/1000,
        "saitaswap": 2/1000,
        "sakeswap_pool": 3/1000,
        "xchange": 200/100000,
        "kingswap": 25/10/1000,
    }

    fee_fun = [
        get_fee_from_fee,
        get_fee_from_swapFee,
        get_fee_from_getSwapFeePercentage,
        get_fee_from_swapStorage,
        get_fee_from_tradingFeePercent,
        get_fee_from_factory
    ]

    if name in fix_fee_dict:
        return fix_fee_dict[name]

    for fun in fee_fun:
        fee = fun(contract, name)
        if fee:
            break
    return fee

In [ ]:
def get_tokens(contract):
    try:
        token0 = contract.functions.token0().call().lower()
        token1 = contract.functions.token1().call().lower()
        return token0, token1
    except Exception as e:
        return None, None

In [ ]:
def get_tvl(address):
    token_type = ['NATIVE']
    payload = {
        "jsonrpc": "2.0",
        "method": "ankr_getAccountBalance",
        "params": {
            "blockchain": ["eth"],
            "walletAddress": address,
            "nativeFirst": True,
            "onlyWhitelisted": True,
            "pageSize": 2
        },
        "id": 1
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json"
    }

    response = requests.post(getAccountBalance_url, json=payload, headers=headers)
    time.sleep(1)
    if response.status_code == 200:
        data = json.loads(response.text)
        tvl = data['result']['totalBalanceUsd']
        return tvl
    print(response)
    return None

In [ ]:
def get_pool_from_events(factory, start_block, end_block):
    pool = []
    name = factory
    for factory_address in factories[factory]["address"]:
        for topic in topics:
            params = {
                "module": "logs",
                "action": "getLogs",
                "fromBlock": start_block,
                "toBlock": end_block,
                "address" : factory_address,
                "topic0": topic,
                "apikey": ETH_API_KEY
            }
            response = requests.post(ETH_URL, params=params)
            time.sleep(0.2)
            data = json.loads(response.text)
            logs = data['result']
            for i in tqdm(range(len(logs))):
                try:
                    log = logs[i]
                    hex_string = log['data']
                    hex_string = hex_string[2:]
                    address = '0x' + hex_string[-40:].lower()

                    contract = get_contract(address)

                    tvl = get_tvl(address)
                    fee = get_fee(contract, name)
                    token0, token1 = get_tokens(contract)
                    row = {
                        "name" : name,
                        "factory" : factory_address,
                        "tvl" : tvl,
                        "fee" : fee,
                        "contract" : address,
                        "token0" : token0,
                        "token1" : token1,
                    }
                    pool.append(row)
                except Exception as e:
                    print(address)
                    continue
    return pool

In [ ]:
def to_file(pool, count):
    pool_df = pd.DataFrame(pool)
    pool_df = pool_df.dropna(axis=0).reset_index(drop=True)

    file_name = f"part{count}.csv"
    pool_df.to_csv(file_name, index=False)

In [ ]:
latest_block = get_latest_block()
latest_block

18593683

In [ ]:
count = 0
for factory in factories.keys():
    interval = factories[factory]["blockInterval"]
    for start in range(0, latest_block, interval):
        end = start + interval
        if end > latest_block:
            end = latest_block
        pool = get_pool_from_events(factory, start, end)
        if len(pool) > 0:
            to_file(pool, count)
            count += 1

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████| 156/156 [06:35<00:00,  2.53s/it]


In [ ]:
files = [f"part{i}.csv" for i in range(count)]

In [ ]:
combined_df = pd.DataFrame()
for file in files:
    df = pd.read_csv(file)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

combined_df = combined_df.drop_duplicates()
combined_df

,name,factory,tvl,fee,contract,token0,token1
0,pancakeswap,0x0bfbcf9fa4f9c56b0f40a671ad40e0805a091865,0.000000e+00,0.0001,0xb078bf211e330b5f95b7114ae845188cc36b795d,0xd84787a01b0cad89fbca231e6960cc0f3f18df34,0xdb19f2052d2b1ad46ed98c66336a5daadeb13005
1,pancakeswap,0x0bfbcf9fa4f9c56b0f40a671ad40e0805a091865,2.074404e-01,0.0100,0xe9825d867e3bef05223bda609fa8ab89aef93797,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xdb19f2052d2b1ad46ed98c66336a5daadeb13005
2,pancakeswap,0x0bfbcf9fa4f9c56b0f40a671ad40e0805a091865,0.000000e+00,0.0005,0x7778797342652bd27b365962ffc7f6ece356eb57,0xd84787a01b0cad89fbca231e6960cc0f3f18df34,0xdb19f2052d2b1ad46ed98c66336a5daadeb13005
3,pancakeswap,0x0bfbcf9fa4f9c56b0f40a671ad40e0805a091865,1.288483e+07,0.0005,0x6ca298d2983ab03aa1da7679389d955a4efee15c,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xdac17f958d2ee523a2206206994597c13d831ec7
4,pancakeswap,0x0bfbcf9fa4f9c56b0f40a671ad40e0805a091865,2.178183e+06,0.0005,0x1ac1a8feaaea1900c4166deeed0c11cc10669d36,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
...,...,...,...,...,...,...,...
151,pancakeswap,0x0bfbcf9fa4f9c56b0f40a671ad40e0805a091865,4.557916e+04,0.0100,0x73b9adc00794260616c51c41997ce0245b3fa012,0xb131f4a55907b10d1f0a50d8ab8fa09ec342cd74,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2
152,pancakeswap,0x0bfbcf9fa4f9c56b0f40a671ad40e0805a091865,1.433232e-01,0.0005,0xd8c6316159727a2dab3e1b8fa25a1d654d56ce9f,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,0xffd418ece52acd0a0a39dbc39baa0c9fe7c90e95
153,pancakeswap,0x0bfbcf9fa4f9c56b0f40a671ad40e0805a091865,2.466740e+02,0.0025,0xb5e9583bcc6a06be4c97c3b6809eda966ac85aa9,0x8b3e6fb1849c9351be45a1e74aa00dec82721913,0xdac17f958d2ee523a2206206994597c13d831ec7
154,pancakeswap,0x0bfbcf9fa4f9c56b0f40a671ad40e0805a091865,8.712738e+04,0.0001,0x3a1b97fc25fa45832f588ed3bfb2a0f74ddbd4f8,0x7f39c581f595b53c5cb19bd0b3f8da6c935e2ca0,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2


In [ ]:
combined_df.to_csv('eth_pancakeswap.csv', index=False)